# NLP: Sentiment Analysis
Choosing and training a model to perform sentiment analysis on catalan text.

In [ ]:
import pandas as pd
import numpy as np

## The training data

Check out `data-preprocessing.ipynb` for details about how this initial dataset was chosen, and how the subset dataset was labeled.

In [1]:
# import subset dataset -> labeled
# import large dataset -> unlabeled

### The manually assigned labels

In [1]:
# Get a list of manually assigned labels

In [ ]:
num_custom_labels = 3

## Fine-tuning

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# Load mBERT model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_custom_labels)